In [1]:
import nltk
import pandas as pd 
nltk.download("gutenberg")
from nltk.corpus import gutenberg
# Load the Gutenberg corpus
data=gutenberg.raw("shakespeare-hamlet.txt")
# Convert the text data into a pandas DataFrame
with open("hamlet.txt", "w") as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     /home/pythonpractical/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
## this is one more file that we can used but in your implementation you can choose any one of them i am 
# usind hamlet.txt
data=gutenberg.raw("shakespeare-macbeth.txt")
# Convert the text data into a pandas DataFrame
with open("macbeth.txt", "w") as file:
    file.write(data)


In [3]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
# Load the text data from the file
with open("hamlet.txt", "r") as file:
    data = file.read().lower()
    
# Tokenize the text data
tokenizer=Tokenizer()
tokenizer.fit_on_texts([data])
total_words=len(tokenizer.word_index)+1

total_words

2025-11-28 15:51:45.278907: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-28 15:51:45.389926: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-28 15:51:49.273801: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


4818

In [4]:
#cretaing a csv file for word index for your better understanding if u want skip this part
df = pd.DataFrame(list(tokenizer.word_index.items()),
                  columns=["word", "index"])
df.to_csv("word_index.csv", index=False)


In [5]:
#creating input sequences
input_sequences=[]
for line in data.split("\n"):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [6]:
input_sequences

[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1, 687, 4, 45, 41, 1886, 1887, 1888],
 [1180, 1889],
 [1180, 1889, 1890],
 [1180, 1889, 1890, 1891],
 [57, 407],
 [57, 407, 2],
 [57, 407, 2, 1181],
 [57, 407, 2, 1181, 177],
 [57, 407, 2, 1181, 177, 1892],
 [407, 1182],
 [407, 1182, 63],
 [408, 162],
 [408, 162, 377],
 [408, 162, 377, 21],
 [408, 162, 377, 21, 247],
 [408, 162, 377, 21, 247, 882],
 [18, 66],
 [451, 224],
 [451, 224, 248],
 [451, 224, 248, 1],
 [451, 224, 248, 1, 30],
 [408, 407],
 [451, 25],
 [408, 6],
 [408, 6, 43],
 [408, 6, 43, 62],
 [408, 6, 43, 62, 1893],
 [408, 6, 43, 62, 1893, 96],
 [408, 6, 43, 62, 1893, 96, 18],
 [408, 6, 43, 62, 1893, 96, 18, 566],
 [451, 71],
 [451, 71, 51],
 [451, 71, 51, 1894],
 [451, 71, 51, 1894, 567],
 [451, 71, 51, 1894, 567, 378],
 [451, 71, 51, 1894, 567, 378, 80],
 [451, 71, 51, 1894, 567, 378, 80, 3],
 [451, 71, 51, 1894, 567, 378, 80, 3, 273],
 [451, 71

In [7]:
# here we can see the input sequences max length u can do like this or else u can assign any value also
max_sequence_len=max([len(x) for x in input_sequences])
max_sequence_len

14

In [8]:
#pad the sequences
input_sequences=pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]],
      shape=(25732, 14), dtype=int32)

In [9]:
#Create predictors and label
import tensorflow as tf
x,y=input_sequences[:,:-1],input_sequences[:,-1]
y

array([ 687,    4,   45, ..., 1047,    4,  193],
      shape=(25732,), dtype=int32)

In [10]:
#we can not let your dependent variable as indexs directl so 
y=tf.keras.utils.to_categorical(y, num_classes=total_words)

In [11]:
#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [12]:
#Train the LSTM Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout,GRU
model=Sequential()
model.add(Embedding(total_words,100, input_length=max_sequence_len-1))
model.add(GRU(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(100))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/home/pythonpractical/miniconda3/envs/myenv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-11-28 15:52:14.378743: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:
history=model.fit(X_train, y_train, epochs=100, verbose=1,validation_data=(X_test, y_test))

Epoch 1/100
  7/644 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - accuracy: 0.0335 - loss: 6.8711  

2025-11-28 16:52:40.476798: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 396714120 exceeds 10% of free system memory.


642/644 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0309 - loss: 6.8796

2025-11-28 16:52:53.523755: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 99192984 exceeds 10% of free system memory.


644/644 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - accuracy: 0.0325 - loss: 6.8524 - val_accuracy: 0.0268 - val_loss: 6.7217
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.0449 - loss: 6.4493 - val_accuracy: 0.0466 - val_loss: 6.7283
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.0572 - loss: 6.1843 - val_accuracy: 0.0600 - val_loss: 6.7047
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.0735 - loss: 5.9068 - val_accuracy: 0.0717 - val_loss: 6.7102
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.0880 - loss: 5.6170 - val_accuracy: 0.0729 - val_loss: 6.7685
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.0976 - loss: 5.3373 - val_accuracy: 0.0711 - val_loss: 6.9053
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.1113 - loss: 5.0575 - val_accuracy: 0.0717 - val_loss: 6.9786
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.1243 - loss: 4.8008 - val_

In [ ]:
# model.save("lstm_text_generator.h5")

In [16]:
# Function to predict the next word
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]  # Ensure the sequence length matches max_sequence_len-1
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [17]:
input_text="To be or not to be"
print(f"Input text:{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next Word PRediction:{next_word}")

Input text:To be or not to be
Next Word PRediction:done


In [18]:
## Save the model
model.save("next_word_GRU.h5")
## Save the tokenizer
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
input_text="Gentlemen within. Hamlet, Lord"
print(f"Input text:{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next Word PRediction:{next_word}")

Input text:Gentlemen within. Hamlet, Lord
Next Word PRediction:hamlet
